### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [1]:
# Check version
version()

In [2]:
# Reset
reset()
# Setup pretty printing
%display latex

In [3]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [4]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

# Components of Metric
grr_ = function('grr_f', latex_name='g_{rr}')(rho, z)
gzz_ = function('gzz_f', latex_name='g_{zz}')(rho, z)
grz_ = function('grz_f', latex_name='g_{rz}')(rho, z)

# Metric is symmetric
G[1, 1], G[2, 2] = grr_, gzz_
G[1, 2] = grz_

# Invert metric
G_con = G.inverse()

# Print metric
G.display()

g = grr_f(rho, z) drho⊗drho + grz_f(rho, z) drho⊗dz + grz_f(rho, z) dz⊗drho + gzz_f(rho, z) dz⊗dz

In [5]:
G_con.display()

inv_g = -gzz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂rho⊗∂/∂rho + grz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂rho⊗∂/∂z + grz_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂z⊗∂/∂rho - grr_f(rho, z)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) ∂/∂z⊗∂/∂z

In [6]:
# Compute Ricci Tensor
R = G.ricci()
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

Nabla.display()

Gam^rho_rho,rho = -1/2*(gzz_f(rho, z)*d(grr_f)/drho + grz_f(rho, z)*d(grr_f)/dz - 2*grz_f(rho, z)*d(grz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_rho,z = -1/2*(gzz_f(rho, z)*d(grr_f)/dz - grz_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_z,rho = -1/2*(gzz_f(rho, z)*d(grr_f)/dz - grz_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^rho_z,z = -1/2*(2*gzz_f(rho, z)*d(grz_f)/dz - gzz_f(rho, z)*d(gzz_f)/drho - grz_f(rho, z)*d(gzz_f)/dz)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_rho,rho = 1/2*(grz_f(rho, z)*d(grr_f)/drho + grr_f(rho, z)*d(grr_f)/dz - 2*grr_f(rho, z)*d(grz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_rho,z = 1/2*(grz_f(rho, z)*d(grr_f)/dz - grr_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_z,rho = 1/2*(grz_f(rho, z)*d(grr_f)/dz - grr_f(rho, z)*d(gzz_f)/drho)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z)) 
Gam^z_z,z = 1/2*(2*grz_f(rho, z)*d(grz_f)/dz - grz_f(rho, z)*d(gzz_f)/drho - grr_f(rho, z)*d(gzz_f)/dz)/(grz_f(rho, z)^2 - grr_f(rho, z)*gzz_f(rho, z))

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [7]:
s_t_ = function('seed_f', latex_name='s')(rho, z)

LamT = M.scalar_field(rho * exp(rho * s_t_) * sqrt(grr_), chart=Chart, name="lam_t", latex_name=r'\lambda')

LamLogT = Nabla(LamT) / LamT
LamLog2T = Nabla(LamLogT) + LamLogT * LamLogT

LamLog2T[1, 1].expr().simplify_full()

-1/4*(rho*grr_f(rho, z)^2*diff(grr_f(rho, z), z)^2 - 4*(rho*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho*grr_f(rho, z)^3*gzz_f(rho, z))*seed_f(rho, z)^2 + (rho*grz_f(rho, z)^2 - 2*rho*grr_f(rho, z)*gzz_f(rho, z))*diff(grr_f(rho, z), rho)^2 - 4*(rho^3*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho^3*grr_f(rho, z)^3*gzz_f(rho, z))*diff(seed_f(rho, z), rho)^2 - 8*(grr_f(rho, z)^2*grz_f(rho, z)^2 - grr_f(rho, z)^3*gzz_f(rho, z))*seed_f(rho, z) - 2*(2*grr_f(rho, z)*grz_f(rho, z)^2 - grr_f(rho, z)^2*gzz_f(rho, z) + (2*rho*grr_f(rho, z)*grz_f(rho, z)^2 - rho*grr_f(rho, z)^2*gzz_f(rho, z))*seed_f(rho, z))*diff(grr_f(rho, z), rho) - 2*(rho*grr_f(rho, z)*grz_f(rho, z)^2 - rho*grr_f(rho, z)^2*gzz_f(rho, z))*diff(grr_f(rho, z), rho, rho) - 2*(rho*grr_f(rho, z)^2*grz_f(rho, z)*seed_f(rho, z) + grr_f(rho, z)^2*grz_f(rho, z))*diff(grr_f(rho, z), z) + 2*(2*rho*grr_f(rho, z)^2*grz_f(rho, z)*seed_f(rho, z) + rho*grr_f(rho, z)*grz_f(rho, z)*diff(grr_f(rho, z), rho) - rho*grr_f(rho, z)^2*diff(grr_f(rho, z), z) + 2*grr_f(rho, z)^2*grz_f(rho, z))*diff(grz_f(rho, z), rho) - 2*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)*diff(grr_f(rho, z), z) - 2*rho^2*grr_f(rho, z)^2*grz_f(rho, z)*diff(grz_f(rho, z), rho) + 8*rho*grr_f(rho, z)^2*grz_f(rho, z)^2 - 8*rho*grr_f(rho, z)^3*gzz_f(rho, z) + 4*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho^2*grr_f(rho, z)^3*gzz_f(rho, z))*seed_f(rho, z) + (2*rho^2*grr_f(rho, z)*grz_f(rho, z)^2 - rho^2*grr_f(rho, z)^2*gzz_f(rho, z))*diff(grr_f(rho, z), rho))*diff(seed_f(rho, z), rho) - 4*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho^2*grr_f(rho, z)^3*gzz_f(rho, z))*diff(seed_f(rho, z), rho, rho) + 2*(rho^2*grr_f(rho, z)^2*grz_f(rho, z)*diff(grr_f(rho, z), rho) + rho^2*grr_f(rho, z)^3*diff(grr_f(rho, z), z) - 2*rho^2*grr_f(rho, z)^3*diff(grz_f(rho, z), rho))*diff(seed_f(rho, z), z))/(rho*grr_f(rho, z)^2*grz_f(rho, z)^2 - rho*grr_f(rho, z)^3*gzz_f(rho, z))

In [8]:
(Nabla(Nabla(LamT)) / LamT)[1, 2].expr().subs({grz_: 0, diff(grr_, rho): 0, diff(gzz_, rho): 0}).expand()

rho^2*diff(seed_f(rho, z), rho)*diff(seed_f(rho, z), z) + rho*seed_f(rho, z)*diff(seed_f(rho, z), z) + rho*diff(seed_f(rho, z), rho, z) + 1/2*diff(grr_f(rho, z), rho, z)/grr_f(rho, z) + 2*diff(seed_f(rho, z), z)

In [9]:
# Seed Function
s_ = function('s_f', latex_name=r's')(rho, z)

# Resulting Tensors
Lam = M.scalar_field(rho * exp(rho * s_) * sqrt(grr_), chart=Chart, name='lam_s', latex_name=r'\lambda')
          
Lam.display()

lam_s: M → ℝ
   (rho, z) ↦ rho*e^(rho*s_f(rho, z))*sqrt(grr_f(rho, z))

In [10]:
# Gauge Fields
lapse_ = function('lapse_f', latex_name=r'\alpha')(rho, z)

shiftr_ = function('shiftr_f', latex_name=r'\beta^r')(rho, z)
shiftz_ = function('shiftz_f', latex_name=r'\beta^z')(rho, z)

# Gauge Tensors
Lapse = M.scalar_field(lapse_, chart=Chart, name='Lapse', latex_name=r'\alpha')

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = shiftr_
Shift.add_comp(Chartf)[2] = shiftz_

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ lapse_f(rho, z)

In [11]:
Shift.display()

Shift = shiftr_f(rho, z) ∂/∂rho + shiftz_f(rho, z) ∂/∂z

In [12]:
# Extrinsic Curvature

# Components
krr_ = function('Krr_f', latex_name='K_{rr}')(rho, z)
kzz_ = function('Kzz_f', latex_name='K_{zz}')(rho, z)
krz_ = function('Krz_f', latex_name='K_{rz}')(rho, z)

# Phi Component of K
y_ = function('y_f', latex_name=r'Y')(rho, z)
# l_ = function('L_f', latex_name='L')(rho, z)

K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = krr_
K.add_comp(Chartf)[2, 2] = kzz_
K.add_comp(Chartf)[1, 2] = krz_

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = G_con['^ij'] * K['_ij']

# L = K_\phi^\phi
L = M.scalar_field(krr_/grr_ + rho * y_, name='L')

K.display()

K = Krr_f(rho, z) drho⊗drho + Krz_f(rho, z) drho⊗dz + Krz_f(rho, z) dz⊗drho + Kzz_f(rho, z) dz⊗dz

In [13]:
L.display()

L: M → ℝ
   (rho, z) ↦ rho*y_f(rho, z) + Krr_f(rho, z)/grr_f(rho, z)

In [14]:
# Z4
theta_ = function('theta_f', latex_name=r'\theta')(rho, z)
Theta = M.scalar_field(theta_, chart=Chart, name='theta_s', latex_name=r'\theta')

zr_ = function('Zr_f', latex_name=r'Z_r')(rho, z)
zz_ = function('Zz_f', latex_name=r'Z_z')(rho, z)

Z = M.tensor_field(0, 1, name='Z')
Z.add_comp(Chartf)[1] = zr_
Z.add_comp(Chartf)[2] = zz_

# Vector form of Z
Zv = G_con['^ij'] * Z['_j']

Theta.display()

theta_s: M → ℝ
   (rho, z) ↦ theta_f(rho, z)

In [15]:
Z.display()

Z = Zr_f(rho, z) drho + Zz_f(rho, z) dz

### Regularization

We want the following equations to be well-defined for both $r > 0$ and $r = 0$. Unfourtunately, to numerically ensure regularity, we must substitute $\lambda \to r e^{r s} \sqrt{g_{rr}}$ and $L = K_{rr} / g_{rr} + rY$. Let 
$$ \Lambda_a = \lambda^{-1} \nabla_a \lambda $$
$$ A_a = \alpha^{-1} \nabla_a \alpha. $$ 
The term $\Lambda_a$ shows up at various points throughout the equations, and is clearly $\mathcal{O}(r^{-1})$ on axis. Taking into account the on-axis behaviour of the other variables, these terms do get cancelled out properly, but Sage does not know this on-axis behaviour (and thus cannot automatically make simplifications like $s/r \to \partial_r s$ when $r = 0$). In order to get around this we define several free variables, and manually make the correct substitution depending on whether $r = 0$.
$$ N = \Lambda_a Z^a $$
$$ O = \Lambda_a A^a $$
$$ P_a = \Lambda_b K_{a}^{\;b}  - \Lambda_a L $$
$$ Q_{ab} = \lambda^{-1} \nabla_a \nabla_b \lambda = \nabla_a \Lambda_b + \Lambda_a \Lambda_b $$

In [16]:
# n_ = function('n_f', latex_name='N')(rho, z)
# o_ = function('o_f', latex_name='O')(rho, z)

# N = M.scalar_field(n_, chart=Chart, name='N')
# O = M.scalar_field(o_, chart=Chart, name='O')

# pr_ = function('pr_f', latex_name="P_r")(rho, z)
# pz_ = function('pz_f', latex_name="P_z")(rho, z)

# P = M.tensor_field(0, 1, sym=(0, 1), name='P')
# P.add_comp(Chartf)[1] = pr_
# P.add_comp(Chartf)[2] = pz_

# qrr_ = function('qrr_f', latex_name='Q_{rr}')(rho, z)
# qrz_ = function('qrz_f', latex_name='Q_{rz}')(rho, z)
# qzz_ = function('qzz_f', latex_name='Q_{zz}')(rho, z)

# Q = M.tensor_field(0, 2, sym=(0, 1), name='Q')
# Q.add_comp(Chartf)[1, 1] = qrr_
# Q.add_comp(Chartf)[2, 2] = qzz_
# Q.add_comp(Chartf)[1, 2] = qrz_

### Constraint equations
Constraint equations that should be satisfied at all times.
$$ \mathcal{C} \equiv \frac{1}{2} (K^2 - K_{ij} K^{ij} + R) - \lambda^{-1} \nabla^j \nabla_j \lambda  + K L = 0 $$

$$ \mathcal{C}_i \equiv \nabla_j K_{i}^{\;j} - \nabla_i (K + L) + \lambda^{-1} (\nabla_j \lambda) K_{i}^{\;j} - \lambda^{-1} (\nabla_i \lambda) L = 0$$

In [17]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - (Nabla(Nabla(Lam)) / Lam)['_ij'] * G_con['^{ij}'] + K_trace * L

# Hamiltonian
CH = term1 + term2

CH

Scalar field on the 2-dimensional differentiable manifold M

In [18]:
# term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
# term2 = LamLog['_j'] * K_mat['^j_i'] - LamLog * L

term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
term2 = (Nabla(Lam) / Lam)['_i'] * K_mat['^i_j'] - (Nabla(Lam) / Lam) * L

# Momentum Constraint
CM = term1 + term2

CM

1-form on the 2-dimensional differentiable manifold M

### Evolution Equations
Calculations for $\mathcal{L}_n X$ for various dynamical quantities.

In [19]:
term1 = R - Nabla(Nabla(Lam)) / Lam - Nabla(Nabla(Lapse)) / Lapse
term2 = (K_trace + L) * K - 2 * K['_ij'] * K_mat['^j_k']
term3 = 2 * Nabla(Z)['_(ij)'] - 2 * K * Theta

# Extrinsic Curvature
LieK = term1 + term2 + term3

In [20]:
term1 = - (Nabla(Nabla(Lam)) / Lam)['_ij'] * G_con['^{ij}']
term2 = - (Nabla(Lam) / Lam * Nabla(Lapse) / Lapse)['_ij'] * G_con['^ij']
term3 = L * (K_trace + L) + 2 * (Nabla(Lam) / Lam)['_i'] * Zv['^i'] - 2 * L * Theta

# Angular Extrinsic Curvature
LieL = term1 + term2 + term3

In [21]:
# Metric and Lambda
LieG = -2*K
LieLam = -Lam * L

In [22]:
term1 = CH + (Nabla(Lam) / Lam - Nabla(Lapse) / Lapse)['_i'] * Zv['^i']
term2 = Nabla(Zv)['^i_i'] - (K_trace + L) * Theta

# Theta
LieTheta = term1 + term2

In [23]:
term1 = CM - 2 * K['_ij'] * Zv['^j']
term2 = - Nabla(Lapse) / Lapse * Theta + Nabla(Theta) 

# Z
LieZ = term1 + term2

In [24]:
# Actual temporal derivatives including shift terms
G_t = Lapse * LieG + G.lie_derivative(Shift)
Lam_t = Lapse * LieLam + Lam.lie_derivative(Shift)
K_t = Lapse * LieK + K.lie_derivative(Shift)
L_t = Lapse * LieL + L.lie_derivative(Shift)

Theta_t = Lapse * LieTheta + Theta.lie_derivative(Shift)
Z_t = Lapse * LieZ + Z.lie_derivative(Shift)

Y_t = (L_t - K_t[1, 1] / G[1, 1] + (G_t[1, 1] * K[1, 1]) / G[1, 1]^2) / rho
s_t = (Lam_t / Lam - G_t[1, 1] / (2 * G[1, 1])) / rho

### Preprocessing
To avoid transcription errors when transferring between notebooks and code we automatically transform symbolic expressions into `C` code.

In [50]:
# Symbolic Variables to replace current functions
var('grr_r grr_z grr_rr grr_zz grr_rz grr')
var('gzz_r gzz_z gzz_rr gzz_zz gzz_rz gzz')
var('grz_r grz_z grz_rr grz_zz grz_rz grz')
var('s_r s_z s_rr s_zz s_rz s')
# var('lam_r lam_z lam_rr lam_zz lam_rz lam')
# var('lam_logr lam_logz')

var('krr_r krr_z krr')
var('kzz_r kzz_z kzz')
var('krz_r krz_z krz')
var('y_r y_z y')
# var('l_r l_z l')

var('lapse_r lapse_z lapse_rr lapse_zz lapse_rz lapse')
var('shiftr_r shiftr_z shiftz_r shiftz_z shiftr shiftz')

var('theta_r theta_z theta')
var('zr_r zr_z zz_r zz_z zr zz')

# Only used for initial data
psi_ = function('psi_f', latex_name=r'\psi')(rho, z)
var('psi_r psi_z psi_rr psi_zz psi_rz psi')

"""
Preprocesses an expression, replacing all derivatives of a function
and invokations of that functions with an appropriately named variable.
"""
def process_expr(expr):
    expr = expr.simplify_full()
    derives = expr.subs({
        diff(grr_, rho): grr_r,
        diff(grr_, z): grr_z,
        diff(grr_, rho, rho): grr_rr,
        diff(grr_, z, z): grr_zz,
        diff(grr_, rho, z): grr_rz,
        diff(grr_, z, rho): grr_rz,
        
        diff(gzz_, rho): gzz_r,
        diff(gzz_, z): gzz_z,
        diff(gzz_, rho, rho): gzz_rr,
        diff(gzz_, z, z): gzz_zz,
        diff(gzz_, rho, z): gzz_rz,
        diff(gzz_, z, rho): gzz_rz,
        
        diff(grz_, rho): grz_r,
        diff(grz_, z): grz_z,
        diff(grz_, rho, rho): grz_rr,
        diff(grz_, z, z): grz_zz,
        diff(grz_, rho, z): grz_rz,
        diff(grz_, z, rho): grz_rz,
        
        diff(krr_, rho): krr_r,
        diff(krr_, z): krr_z,
        
        diff(kzz_, rho): kzz_r,
        diff(kzz_, z): kzz_z,
        
        diff(krz_, rho): krz_r,
        diff(krz_, z): krz_z,
        
        diff(s_, rho): s_r,
        diff(s_, z): s_z,
        diff(s_, rho, rho): s_rr,
        diff(s_, z, z): s_zz,
        diff(s_, rho, z): s_rz,
        diff(s_, z, rho): s_rz,
        
        diff(lapse_, rho): lapse_r,
        diff(lapse_, z): lapse_z,
        diff(lapse_, rho, rho): lapse_rr,
        diff(lapse_, z, z): lapse_zz,
        diff(lapse_, rho, z): lapse_rz,
        diff(lapse_, z, rho): lapse_rz,
        
        diff(shiftr_, rho): shiftr_r,
        diff(shiftr_, z): shiftr_z,
        diff(shiftz_, rho): shiftz_r,
        diff(shiftz_, z): shiftz_z,
        
        diff(y_, rho): y_r,
        diff(y_, z): y_z,
        
        diff(theta_, rho): theta_r,
        diff(theta_, z): theta_z,
        
        diff(zr_, rho): zr_r,
        diff(zr_, z): zr_z,
        diff(zz_, rho): zz_r,
        diff(zz_, z): zz_z,
    })
    
    values = derives.subs({
        grr_: grr,
        gzz_: gzz,
        grz_: grz,
        s_: s,
        
        krr_: krr,
        kzz_: kzz,
        krz_: krz,
        y_: y,

        lapse_: lapse,
        shiftr_: shiftr,
        shiftz_: shiftz,
        
        theta_: theta,
        zr_: zr,
        zz_: zz,
    })
    
    return values

def process_initial_expr(expr):
    expr = expr.simplify_full()
    expr = expr.subs({
        diff(grr_, rho): diff(psi_^4, rho).expand(),
        diff(grr_, z): diff(psi_^4, z).expand(),
        diff(grr_, rho, rho): diff(psi_^4, rho, rho).expand(),
        diff(grr_, z, z): diff(psi_^4, z, z).expand(),
        
        diff(gzz_, rho): diff(psi_^4, rho).expand(),
        diff(gzz_, z): diff(psi_^4, z).expand(),
        diff(gzz_, rho, rho): diff(psi_^4, rho, rho).expand(),
        diff(gzz_, z, z): diff(psi_^4, z, z).expand(),
        
        diff(grz_, rho): SR(0),
        diff(grz_, z): SR(0),
        diff(grz_, rho, rho): SR(0),
        diff(grz_, z, z): SR(0),
        diff(grz_, rho, z): SR(0),
        
        grr_: psi_^4,
        grz_: SR(0),
        gzz_: psi_^4,
        
        krr_: SR(0),
        kzz_: SR(0),
        krz_: SR(0),
        y_: SR(0),
        
        shiftr_: SR(0),
        shiftz_: SR(0),
        
        theta_: SR(0),
        zr_: SR(0),
        zz_: SR(0),
    }).expand()
    
    derives = expr.subs({
        diff(psi_, rho): psi_r,
        diff(psi_, z): psi_z,
        diff(psi_, rho, rho): psi_rr,
        diff(psi_, z, z): psi_zz,
        diff(psi_, rho, z): psi_rz,
        diff(psi_, z, rho): psi_rz,
        
        diff(s_, rho): s_r,
        diff(s_, z): s_z,
        diff(s_, rho, rho): s_rr,
        diff(s_, z, z): s_zz,
        diff(s_, rho, z): s_rz,
        diff(s_, z, rho): s_rz,
        
        diff(lapse_, rho): lapse_r,
        diff(lapse_, z): lapse_z,
        diff(lapse_, rho, rho): lapse_rr,
        diff(lapse_, z, z): lapse_zz,
        diff(lapse_, rho, z): lapse_rz,
        diff(lapse_, z, rho): lapse_rz,
    })
    
    values = derives.subs({
        psi_: psi,
        s_: s,

        lapse_: lapse,
    })
    
    return values

diff(psi_^4, rho, rho).expand()

12*psi_f(rho, z)^2*diff(psi_f(rho, z), rho)^2 + 4*psi_f(rho, z)^3*diff(psi_f(rho, z), rho, rho)

In [60]:
def regularize(expr):
    sym_expr = expr._sympy_().expand()
    sym_expr = sym_expr.expand().subs(grz / rho, grz_r)
    sym_expr = sym_expr.expand().subs(grz, 0)
    sym_expr = sym_expr.expand().subs([
        (grr_r / rho, grr_rr),
        (gzz_r / rho, gzz_rr),
        (krz / rho, krz_r),
        (lapse_r / rho, lapse_rr),
        (s / rho, s_r),
        (zr / rho, zr_r),
        (psi_r / rho, psi_rr),
    ])
    sym_expr = sym_expr.expand().subs([
        (grz_z, 0),
    ])
    # We should have removed all 1/r terms. If we didn't, there are now infs
    sym_expr = sym_expr.expand().subs(rho, 0)
    sym_expr = sym_expr.expand().subs([
        (grr_r, 0),
        (gzz_r, 0),
        (krz, 0),
        (krr_r, 0),
        (kzz_r, 0),
        (s, 0),
        (y, 0),
        (lapse_r, 0),
        (shiftr, 0),
        (shiftz_r, 0),
        (theta_r, 0),
        (zr, 0),
        (zz_r, 0),
        
        (psi_r, 0),
    ])
    sym_expr = sym_expr.expand().subs([
        (grr_rz, 0),
        (gzz_rz, 0),
        (grz_z, 0),
        (grz_rz, 0),
        (s_z, 0),
        (s_rz, 0),
        (krz_z, 0),
        (y_z, 0),
        (zr_z, 0),
        (lapse_rz, 0),
        (shiftr_z, 0),
    ])
    
    return sym_expr.expand()._sage_()

# Term = (Nabla(Lam) / Lam)['_i'] * K_mat['^i_j'] - Nabla(Lam) / Lam * L

# expr = process_expr(Term[1].expr()).expand()
# expr

In [61]:
import sympy as sym

def generate_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(process_expr(eq)._sympy_().simplify())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

def generate_regular_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(regularize(process_expr(eq))._sympy_().simplify())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

In [67]:
import sympy as sym

def generate_initial_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(process_initial_expr(eq)._sympy_().simplify())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

def generate_initial_regular_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(regularize(process_initial_expr(eq))._sympy_().expand())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

In [68]:
hyperbolic = [
    ("grr_t", G_t[1, 1].expr()),
    ("gzz_t", G_t[2, 2].expr()),
    ("grz_t", G_t[1, 2].expr()),
    ("krr_t", K_t[1, 1].expr()),
    ("kzz_t", K_t[2, 2].expr()),
    ("krz_t", K_t[1, 2].expr()),
    ("theta_t", Theta_t.expr()),
    ("zr_t", Z_t[1].expr()),
    ("zz_t", Z_t[2].expr()),
    ("y_t", Y_t.expr()),
    ("s_t", s_t.expr()),
]

code = generate_ccode(hyperbolic)

hyperbolic_h = open("hyperbolic.h", 'w')
hyperbolic_h.write(code)
hyperbolic_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = 2*grr;
double x1 = 2*grz;
double x2 = 2*lapse;
double x3 = krr*x2;
double x4 = 2*gzz;
double x5 = krz*x2;
double x6 = pow(grr, 4);
double x7 = pow(gzz_r, 2);
double x8 = lapse*rho;
double x9 = (1.0/4.0)*x8;
double x10 = x6*x7*x9;
double x11 = pow(grr, 2);
double x12 = pow(grz, 2);
double x13 = x11*x12;
double x14 = pow(grr_z, 2);
double x15 = x14*x9;
double x16 = grr*gzz;
double x17 = -x12;
double x18 = x16 + x17;
double x19 = pow(grr, 3);
double x20 = rho*x19;
double x21 = lapse*x20;
double x22 = x18*x21;
double x23 = grz_rz*x22;
double x24 = (1.0/2.0)*x22;
double x25 = gzz_rr*x24;
double x26 = grz_z*x21;
double x27 = grr_r*gzz;
double x28 = grr_z*grz;
double x29 = grz_r*x1;
double x30 = x27 + x28 - x29;
double x31 = (1.0/2.0)*x30;
double x32 = grr*grr_z;
double x33 = grr_r*grz;
double x34 = -grz_r*x0 + x32 + x33;


In [69]:
hyperbolic_regular = [
    ("grr_t", G_t[1, 1].expr()),
    ("gzz_t", G_t[2, 2].expr()),
    ("grz_t", G_t[1, 2].expr()),
    ("krr_t", K_t[1, 1].expr()),
    ("kzz_t", K_t[2, 2].expr()),
    ("krz_t", K_t[1, 2].expr()),
    ("theta_t", Theta_t.expr()),
    ("zr_t", Z_t[1].expr()),
    ("zz_t", Z_t[2].expr()),
    ("y_t", SR(0)),
    ("s_t", SR(0)),
]

code = generate_regular_ccode(hyperbolic_regular)

hyperbolic_regular_h = open("hyperbolic_regular.h", 'w')
hyperbolic_regular_h.write(code)
hyperbolic_regular_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = 2*shiftr_r;
double x1 = 2*lapse;
double x2 = krr*x1;
double x3 = 2*shiftz_z;
double x4 = kzz*x1;
double x5 = pow(gzz, 2);
double x6 = 1.0/x5;
double x7 = gzz*lapse;
double x8 = grr_z*x7;
double x9 = lapse*x5;
double x10 = 2*grz_r;
double x11 = gzz_z*lapse;
double x12 = krr*theta;
double x13 = grr*x5;
double x14 = grr_z*zz;
double x15 = grz_r*zz;
double x16 = grr*gzz;
double x17 = grr*(grr_z - grz_r);
double x18 = x11*x17;
double x19 = -grr_zz - gzz_rr + 2*krr*kzz;
double x20 = x1*x16;
double x21 = pow(grr, 2);
double x22 = gzz_z*zz;
double x23 = lapse*zz_z;
double x24 = gzz*x21;
double x25 = 1.0/x21;
double x26 = x25/gzz;
double x27 = kzz*lapse;

// Final Equations
double grr_t = grr*x0 + grr_z*shiftz - x2;

double gzz_t = gzz*x3 + gzz_z*shiftz - x4;

double grz_t = 0;

double krr_t = x6*((1.0/4.0)*grr*x11*(grr_z - 

In [70]:
# constraints = [
#     ("hamiltonian", CH.expr()),
#     ("momentum_r", CM[1].expr()),
#     ("momentum_z", CM[2].expr()),
# ]

# code = generate_ccode(constraints)

# constaints_h = open("constraints.h", 'w')
# constaints_h.write(code)
# constaints_h.close()

# print(code)

In [71]:
# code = generate_regular_ccode(constraints)

# constaints_regular_h = open("constraints_regular.h", 'w')
# constaints_regular_h.write(code)
# constaints_regular_h.close()

# print(code)

In [72]:
code = generate_initial_ccode([
    ("op", CH.expr()),
])

initial_h = open("initial.h", 'w')
initial_h.write(code)
initial_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = 2*s;
double x1 = rho*s_r;
double x2 = pow(rho, 2);

// Final Equations
double op = -(psi*rho*(rho*s_rr + rho*s_zz + pow(s, 2) + pow(s_r, 2)*x2 + 4*s_r + pow(s_z, 2)*x2 + x0*x1) + psi*x0 + 4*psi_r + 4*rho*(psi_r*s + psi_r*x1 + psi_rr + psi_z*rho*s_z + psi_zz))/(pow(psi, 5)*rho);




In [73]:
code = generate_initial_regular_ccode([
    ("op", CH.expr()),
])

initial_h = open("initial_regular.h", 'w')
initial_h.write(code)
initial_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = pow(psi, -5);

// Final Equations
double op = -8*psi_rr*x0 - 4*psi_zz*x0 - 6*s_r/pow(psi, 4);


